# **EXPORTAÇÃO E ANÁLISE DA DATABASE**

In [ ]:
#Configurando bibliotecas e ambiente de execução
!pip install spotipy
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Criar dataframe
treino = pd.read_csv("musicas_categorizadas.csv")

In [ ]:
#Obter informações sobre a database

print("A database \'musicas_categorizadas\' possui as seguintes colunas: \n \n ", treino.columns , "\n--------------------------------------")
print("Cada coluna da database \'musicas_categorizadas\' armazena os seguintes tipos de dados: \n \n", treino.dtypes , "\n--------------------------------------")
print("Cada coluna da database \'musicas_categorizadas\' possui a seguinte soma de valores nulos: \n \n", treino.isnull().sum(), "\n--------------------------------------")

duplicados = treino.duplicated()
linhas_duplicadas = treino[treino.duplicated()]

print("Por fim, há", linhas_duplicadas.shape[0], "linhas duplicadas na database \'musicas_categorizadas.csv\'.")

In [ ]:
#Verificar a frequência de gêneros na database
frequencies = treino['Class'].value_counts()

print("Cada gênero presente no database aparece no seguinte número de vezes: \n \n", frequencies)

# **TRANSFORMAÇÕES NA DATABASE**

In [ ]:
#Removendo as colunas que não influenciam o modelo
treino = treino.drop(['Artist Name', 'Track Name', 'Popularity', 'duration_in min/ms'], axis=1)
print("As colunas restantes são: \n", treino.columns)

In [ ]:
#Verificando valores nulos nas colunas restantes
print(treino.isnull().sum())

In [ ]:
#Imputando dados em 'key' a partir da abordagem da WEB api Spotify - NULOS = -1
treino['key'] = treino['key'].fillna(-1)

In [ ]:
#Limitando a coluna 'instrumentalness' para 4 casas decimais
treino['instrumentalness'] = treino['instrumentalness'].round(4)

#Imputando dados em 'instrumentalness' com a média de cada gênero
for i in range(11):
  media = treino.loc[treino['Class'] == i, 'instrumentalness'].mean()
  media = media.round(4)
  print("Gênero", i, "tem média", media)
  treino.loc[(treino['Class'] == i) & (treino['instrumentalness'].isna()), 'instrumentalness'] = media

#Transformando 'tempo' em inteiro (BPM)
treino['tempo'] = treino['tempo'].round()

In [ ]:
#Verificando se preenchimento aconteceu com sucesso
print(treino.isnull().sum())

# **TREINAMENTO DOS MODELOS DE IA**

In [ ]:
#Desenvolvendo modelo de ML
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Separando as características e o alvo
X = treino.drop('Class', axis=1)
y = treino['Class']


#REDE NEURAL ------------------------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Criando o modelo de rede neural com os parâmetros fornecidos
clf_neural = MLPClassifier(hidden_layer_sizes=(50, 50, 50, 50), activation='tanh', solver='adam', alpha=0.05, learning_rate='constant', max_iter=500, random_state=42)

#Treinando o modelo
clf_neural.fit(X_train, y_train)

#Fazendo previsões com o conjunto de teste
y_pred = clf_neural.predict(X_test)

#Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)

print(f'Matriz de Confusão da Rede Neural \n')

confusion_matrix(y_test, y_pred)

print(f'\n Acurácia do modelo de rede neural é : {accuracy:.2f}% \n -------------------')

#------------------------------------------------------------

# ARVORE DE DECISÃO ------------------------------------------------------------
from sklearn.tree import DecisionTreeClassifier

#Criando o modelo de árvore de decisão
clf_tree = DecisionTreeClassifier(criterion = 'gini', splitter = 'best', max_depth = 10, min_samples_leaf = 1, min_samples_split = 10, random_state = 42)

#Treinando o modelo
clf_tree.fit(X_train, y_train)

#Fazendo previsões com o conjunto de teste
y_pred = clf_tree.predict(X_test)

#Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)

print(f'Matriz de Confusão da Árvore de Decisâo \n')

confusion_matrix(y_test, y_pred)

print(f'Acurácia do modelo de arvore de decisão: {accuracy:.2f}& \n')

#------------------------------------------------------------

# NAIVE BAYES (Bayes Inocente) ------------------------------------------------------------

from sklearn.naive_bayes import GaussianNB

# Criando o modelo Naive Bayes
clf_nb = GaussianNB()

# Treinando o modelo
clf_nb.fit(X_train, y_train)

# Fazendo previsões com o conjunto de teste
y_pred = clf_nb.predict(X_test)

# Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)

print(f'Matriz de Confusão do Naive Bayes \n')

confusion_matrix(y_test, y_pred)

print(f'Acurácia do modelo Naive Bayes: {accuracy:.2f}%')



# **INTERAÇÃO DO USUÁRIO - PREVENDO GÊNERO DE MÚSICA NOVA**

In [ ]:
#Credenciais da API
client_credentials_manager = SpotifyClientCredentials(client_id='9b1e9ff837434408ae773f93a6facb78', client_secret='67dbfcb123f343b2a6921dbfe243914f')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#Obtendo o nome da música do usuário
nome_da_musica = input("Digite o nome da música que você quer pesquisar: ")
results = sp.search(q=nome_da_musica, limit=1)
musica = results['tracks']['items'][0]['name']
autor = results['tracks']['items'][0]['artists'][0]['name']

#Mostrando o resultado
print("A música que você buscou retornou o resultado: ", musica, " - ", autor)

print("\nOuça uma prévia da música que você selecionou: ")
id_da_musica = results['tracks']['items'][0]['id']
embed_link = f"https://open.spotify.com/embed/track/{id_da_musica}"
iframe = f"""<iframe src="{embed_link}" width="380" height="200" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>"""
display(HTML(iframe))


#Obtendo os dados da música
dadosDaMusica = sp.audio_features(id_da_musica)[0]

#Limitando o atributo 'instrumentalness' para apenas 4 casas decimais
dadosDaMusica['instrumentalness'] = round(dadosDaMusica['instrumentalness'], 4)

# Mapeando os dados fornecidos para a ordem desejada
caracteristicas_relevantes = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature']
valores = [dadosDaMusica[key] for key in caracteristicas_relevantes]

#Selecionando o modelo de IA que o usuário quer usar
print("Qual modelo você quer usar pra predição da sua música? \n \t  1 - Árvore de Decisão \n \t  2 - Rede Neural \n \t  3 - Naive Bayes")
modelo = input()

# Mapeando a entrada do usuário para o modelo correspondente
modelos = {'1': clf_tree, '2': clf_neural, '3': clf_nb}

# Verificando se a entrada do usuário é válida
if modelo in modelos:
    # Fazendo a previsão com o modelo escolhido
    predicao = modelos[modelo].predict([valores])
else:
    print("Entrada inválida. Por favor, escolha 1, 2 ou 3.")

if predicao == 0:
    categoria_correspondente = "Acoustic/Folk"
elif predicao == 1:
    categoria_correspondente = "Alt Music"
elif predicao == 2:
    categoria_correspondente = "Blues"
elif predicao == 3:
    categoria_correspondente = "Bollywood"
elif predicao == 4:
    categoria_correspondente = "Country"
elif predicao == 5:
    categoria_correspondente = "HipHop"
elif predicao == 6:
    categoria_correspondente = "Indie/Alt"
elif predicao == 7:
    categoria_correspondente = "Instrumental"
elif predicao == 8:
    categoria_correspondente = "Metal"
elif predicao == 9:
    categoria_correspondente = "Pop"
elif predicao == 10:
    categoria_correspondente = "Rock"
else:
    categoria_correspondente = "Categoria Desconhecida"

#Printando resultado
print("-------------------------------------- \n A musica", musica, "de", autor, "faz parte do Gênero Musical -", categoria_correspondente, predicao)

#Obtendo recomendações
recommendations = sp.recommendations(seed_tracks=[id_da_musica],
                                      limit=10,
                                      target_danceability=dadosDaMusica['danceability'],
                                      target_energy=dadosDaMusica['energy'],
                                      target_liveness=dadosDaMusica['mode'],
                                      target_valence=dadosDaMusica['valence'])


# Imprimir os nomes das músicas recomendadas e seus artistas
print("-------------------------------------- \n Músicas com a mesma ｖ   ｉ   ｂ   ｅ: \n ")
for track in recommendations['tracks'][1:]:
    print(f"\t Música: {track['name']} — {track['artists'][0]['name']} \n")
